In [15]:
import sys
import pandas as pd
from glob import glob
from tqdm import tqdm   

sys.path.append('../')

In [12]:
from src.tacogfn.data.pharmacophore import PharmacoDB
from src.scoring.scoring_module import AffinityPredictor

In [13]:
HEAD_PATH = "../model_weights/base_100_per_pocket.pth"
device = 'cuda'
predictor = AffinityPredictor(HEAD_PATH, device)

In [14]:
pharmacophore_paths = glob('../dataset/docking_pharmacophores/*.pt')

In [17]:
zinc250k = pd.read_csv('../dataset/zinc_250k.csv')

In [33]:
smiles_list = [smi.strip() for smi in zinc250k['smiles'].sample(20).to_list()]

In [34]:
avg_scores = {}

In [35]:
import torch
import numpy as np

for pharmacophore_path in tqdm(pharmacophore_paths):
    pocket_id = pharmacophore_path.split('/')[-1].split('.')[0]
    features = torch.load(pharmacophore_path, map_location=device)
    
    avg_scores[pocket_id] = []
    for smiles in smiles_list:
        score = predictor.scoring(features, smiles).item()
        avg_scores[pocket_id].append(score)

100%|██████████| 15252/15252 [12:07<00:00, 20.95it/s]


In [37]:
final_scores = {}
for pocket_id in avg_scores.keys():
    final_scores[pocket_id[:-4]] = np.mean(avg_scores[pocket_id])

In [39]:
torch.save(final_scores, '../model_weights/avg_scores/avg_for_base_100_per_pocket.pt')